In [253]:
#!/usr/bin/env python3
"""
Module Docstring
"""

__author__ = "Your Name"
__version__ = "0.1.0"
__license__ = "MIT"

import argparse
import os
from logzero import logger
from prettytable import PrettyTable
from espn_api.football import League


POSITIONS = ["QB", "WR", "TE", "RB"]

""" Main entry point of the app """

# Get environment variables
swid = os.getenv('SWID')
espn_s2 = os.environ.get('ESPN_S2')

if swid == None or espn_s2 == None:
    print("SWID or ESPN_S2 environment variables empty. Exiting...")



## Get Box Scores, pass in week 
week = 2
# if args.week != None:
#     week = args.week
# for week in range(week):
league = League(league_id=761056, year=2022, espn_s2=espn_s2,swid=swid)

print("Week: ", week)
# box_scores = league.box_scores(2)
# for matchup in box_scores:
#     print(matchup.home_team.team_name )

Week:  2


In [287]:
def biggestBlowOut(league, week):
    outcomes = []
    
    box_scores = league.box_scores(week)
    
    for matchup in box_scores:
        ## For Each Matchip calculate the diff
        diff = 0
        if matchup.home_score >= matchup.away_score:
            diff = matchup.home_score - matchup.away_score
            outcomes.append((matchup, diff))
        else:
            diff = matchup.away_score - matchup.home_score
            outcomes.append((matchup, diff))
            
    biggest_blowout, diff = sorted(outcomes, key=lambda tup: tup[1], reverse=True)[0]
    
    
    
    j = PrettyTable()
#     title = "Biggest Blowout ({0})".format(diff)
    title = "Biggest Blowout".format(diff)
    j.title = title
    j.field_names=[biggest_blowout.home_team.team_name,biggest_blowout.away_team.team_name]
    j.add_row([biggest_blowout.home_score, biggest_blowout.away_score])
    j.align="c"
    print(j)

    return


blowoutMatchup = biggestBlowOut(league, week)

+----------------------------------+
|         Biggest Blowout          |
+-----------------------+----------+
| Kupp My  Bells Gurley | El Chapo |
+-----------------------+----------+
|         119.2         |  161.9   |
+-----------------------+----------+


In [254]:
def biggestBenchWarmer(league, week, position):
    if position not in POSITIONS:
        return []
    
    
    benchWarmers = []
    box_scores = league.box_scores(week)
    for matchup in box_scores:
        ## For Each Matchip calculate the diff
        for player in matchup.home_lineup:
            if player.position == position and player.slot_position == "BE":
#                 print("player: ", player.name, " position: ", player.position, " ", player.slot_position, " ", matchup.home_team, " ", player.points)
                benchWarmers.append((player.points,player.name,player.position,matchup.home_team.team_name))
                
                
        for player in matchup.away_lineup:
            if player.position == position and player.slot_position == "BE":
#                 print("player: ", player.name, " position: ", player.position, " ", player.slot_position, " ", matchup.away_team, " ",player.points )
                benchWarmers.append((player.points,player.name,player.position,matchup.away_team.team_name))
    

    return sorted(benchWarmers, key=lambda tup: tup[0], reverse=True)


def prettyPrintBenchWarmers(benchWarmers, position):
    x = PrettyTable()
    x.field_names = [" ", "Points", "Player", "Team"]
    x.title = position + ' Benchwarmers'
    for index, warmer in enumerate(benchWarmers):
        x.add_row([index+1,warmer[0],warmer[1],warmer[3]])
    print(x)
    print("  ")
        

### QBs
list_len = 3
qbWarmers = biggestBenchWarmer(league, week, "QB")
prettyPrintBenchWarmers(qbWarmers[:list_len], "QB")

### RBs
rbWarmers = biggestBenchWarmer(league, week, "RB")
prettyPrintBenchWarmers(rbWarmers[:list_len], "RB")

### TEs
teWarmers = biggestBenchWarmer(league, week, "TE")
prettyPrintBenchWarmers(teWarmers[:list_len], "TE")


#### WRs
teWarmers = biggestBenchWarmer(league, week, "WR")
prettyPrintBenchWarmers(teWarmers[:list_len], "WR")
        
    

+-----------------------------------------------------+
|                   QB Benchwarmers                   |
+---+--------+----------------+-----------------------+
|   | Points |     Player     |          Team         |
+---+--------+----------------+-----------------------+
| 1 |  26.7  |  Carson Wentz  | Gisele'd in my  Wentz |
| 2 |  14.4  | Marcus Mariota |     Rosen Sandler     |
| 3 |  12.6  |   Mac Jones    |     The  Red Army     |
+---+--------+----------------+-----------------------+
  
+-------------------------------------------------------+
|                    RB Benchwarmers                    |
+---+--------+------------------+-----------------------+
|   | Points |      Player      |          Team         |
+---+--------+------------------+-----------------------+
| 1 |  17.8  |   Tony Pollard   |         Top G         |
| 2 |  15.7  |  Damien Harris   |   Chiraqui Soldiers   |
| 3 |  14.6  | David Montgomery | Kupp My  Bells Gurley |
+---+--------+---------------

In [277]:
# ### Total Points Played / Best Possible Points
def optimalLineup(lineup):  

    QB = 2
    RB = 2
    WR_TE = 3
    FLEX = 1

    starters = []
    starter_ouput = 0
    all_players = []
    
    optimal_players = []
    optimal_output = 0
    
    for player in lineup:
        if player.slot_position != "IR" and player.slot_position != "BE":
            starters.append(player) 
            starter_ouput += player.points

    all_players = sorted(lineup, key=lambda x: x.points, reverse=True)


    ## Get Best 2QBs
    for player in all_players:
        if QB > 0 and player.position == "QB":
                QB = QB - 1
                optimal_players.append(player)
                optimal_output += player.points

            
    ## Get Single FLEX Score
    for player in all_players:
        if FLEX > 0 and  (player.position == "RB" or player.position == "WR" or player.position == "TE") and (player not in optimal_players):
            FLEX = FLEX - 1
            optimal_players.append(player)
            optimal_output += player.points
     
    ## Get Best 2 RBs
    for player in all_players:
        if RB > 0 and (player.position == "RB") and (player not in optimal_players):
            RB = RB -1
            optimal_players.append(player)
            optimal_output += player.points
    
    ## Get Best WR/TE
    for player in all_players:
        if   WR_TE > 0 and (player.position == "TE" or player.position == "WR") and (player not in optimal_players):
            WR_TE = WR_TE - 1
            optimal_players.append(player)
            optimal_output += player.points
    
    return starter_ouput, optimal_output
 
    
def manager_effiency(league, week):
    manager_eff = []
    for matchup in league.box_scores(week):
            starter_ouput,optimal_output = optimalLineup(matchup.home_lineup)
            manager_eff.append((matchup.home_team.team_name,starter_ouput,optimal_output,starter_ouput/optimal_output))

            starter_ouput,optimal_output = optimalLineup(matchup.away_lineup)
            manager_eff.append((matchup.away_team.team_name,starter_ouput,optimal_output, starter_ouput/optimal_output))
            manager_eff = sorted(manager_eff, key=lambda manager_eff: manager_eff[3], reverse=True)
    return manager_eff


def prettyPrintManagerEff(manager_eff):
    
    x = PrettyTable()
    x.title = ' Smartest Managers of the Week'
    x.field_names = ["Team", "Output/Optimal", "%"]

    manager_eff_sorted = sorted(manager_eff, key=lambda manager_eff: manager_eff[3], reverse=True)
    for manager in manager_eff_sorted[:2]:
        fraction = "{0:.0f}/{1:.0f}".format(manager[1],manager[2]) 
        percentage = "{0:.0f}".format(manager[3] * 100)
        x.add_row([manager[0], fraction,percentage])
    print(x)
    
    y = PrettyTable()
    y.title = ' Worst Managers of the Week'
    y.field_names = ["Team", "Output/Optimal", "%"]

    worst_manager_eff_sorted = sorted(manager_eff, key=lambda manager_eff: manager_eff[3], reverse=False)
    for manager in worst_manager_eff_sorted[:2]:
#         print("{0}: {1:.2f}/{2:.2f}  {3:4.2f}%".format(manager[0], manager[1],manager[2],manager[3]*100))
        fraction = "{0:.0f}/{1:.0f}".format(manager[1],manager[2]) 
        percentage = "{0:.0f}".format(manager[3] * 100)
        y.add_row([manager[0], fraction,percentage])
    print(y)

manager_eff = manager_effiency(league, week)
prettyPrintManagerEff(manager_eff)

+--------------------------------------+
|     Smartest Managers of the Week    |
+----------------+----------------+----+
|      Team      | Output/Optimal | %  |
+----------------+----------------+----+
| Grayson BeatYu |    123/124     | 99 |
| The  Red Army  |     84/85      | 98 |
+----------------+----------------+----+
+-----------------------------------------+
|        Worst Managers of the Week       |
+-------------------+----------------+----+
|        Team       | Output/Optimal | %  |
+-------------------+----------------+----+
| Chiraqui Soldiers |     81/115     | 70 |
|   J.G.'s  ZZ's    |     90/128     | 70 |
+-------------------+----------------+----+


In [286]:
### Season Long Mananger Effenciey returns -> sorted list(team_name, total_output, optimal_output, accuracy %)

def seasonEffiency(league, week):
    weekly_eff = {}
    output_eff = []
    ## For Each Week Gather the Effenciey
    for i in range(1,week+1):
#         print("Week: ", i)
        weekly_eff[i] = {}
        for matchup in league.box_scores(i):
            
            starter_ouput,optimal_output = optimalLineup(matchup.home_lineup)
            weekly_eff[i][matchup.home_team.team_name] = [starter_ouput,optimal_output,starter_ouput/optimal_output]

            starter_ouput,optimal_output = optimalLineup(matchup.away_lineup)
            weekly_eff[i][matchup.away_team.team_name] = [starter_ouput,optimal_output,starter_ouput/optimal_output]
#           manager_eff.append((matchup.away_team.team_name,starter_ouput,optimal_output, starter_ouput/optimal_output))

    ## For each week for each team
    for team in league.teams:
        output = 0
        optimal = 0
        accuracy = []
        for i in range(1,week+1):
#             print(team.team_name, " ", weekly_eff[i][team.team_name])
            output += weekly_eff[i][team.team_name][0]
            optimal += weekly_eff[i][team.team_name][1]
            accuracy.append(weekly_eff[i][team.team_name][2])
            
        
        output_eff.append((team.team_name, output,optimal,sum(accuracy)/len(accuracy)))
        
    output_eff = sorted(output_eff, key=lambda output_eff: output_eff[3], reverse=True)
    return output_eff

def prettyPrintSeasonEff(total_season_eff):
    
    y = PrettyTable()
    y.title = "Total Manager Efficiency Rankings"
    y.field_names = ["Team", "Output/Optimal", "%"]
    for team in total_season_eff: 
        fraction = "{0:.0f}/{1:.0f}".format(team[1],team[2]) 
        percentage = "{0:.0f}".format(team[3] * 100)
        y.add_row([team[0], fraction,percentage])
    print(y)

total_season_eff = seasonEffiency(league, week)
prettyPrintSeasonEff(total_season_eff)


        

+---------------------------------------------+
|      Total Manager Efficiency Rankings      |
+-----------------------+----------------+----+
|          Team         | Output/Optimal | %  |
+-----------------------+----------------+----+
|        El Chapo       |    262/279     | 94 |
|     The  Red Army     |    194/209     | 94 |
| Kupp My  Bells Gurley |    248/266     | 93 |
|     Grayson BeatYu    |    253/283     | 91 |
|    Begelbaum Donuts   |    213/243     | 88 |
|         Top G         |    215/248     | 87 |
|     J.G.'s  ZZ's      |    206/247     | 84 |
|     Rosen Sandler     |    216/273     | 79 |
| Gisele'd in my  Wentz |    205/272     | 74 |
|   Chiraqui Soldiers   |    171/253     | 68 |
+-----------------------+----------------+----+


In [180]:
def worstWin(league, week):
    box_scores = league.box_scores(week)
    lowestWinningScore = 99999999999
    saved = None
    for matchup in box_scores:
        if matchup.home_score >= matchup.away_score:
            lowestWinningScore = min(matchup.home_score, lowestWinningScore)
        else:
            lowestWinningScore = min(matchup.away_score, lowestWinningScore)
            
        
        if lowestWinningScore == matchup.away_score or lowestWinningScore == matchup.home_score:
            saved = matchup
         
    j = PrettyTable()
    j.title = "Worst Win"
    j.field_names=[saved.home_team.team_name,saved.away_team.team_name]
    j.add_row([saved.home_score, saved.away_score])
    j.align="c"
    print(j)
            

    
worstWin(league, week)

+-------------------------------+
|           Worst Win           |
+---------------+---------------+
| J.G.'s  ZZ's  | The  Red Army |
+---------------+---------------+
|      89.8     |      83.6     |
+---------------+---------------+


In [182]:
def worstLoss(league, week):
    box_scores = league.box_scores(week)
    higestLoss = 0
    saved = None
    for matchup in box_scores:
        if matchup.home_score >= matchup.away_score:
            higestLoss = max(matchup.away_score, higestLoss)
        else:
            higestLoss = max(matchup.home_score, higestLoss)
            
        
        if higestLoss == matchup.away_score or higestLoss == matchup.home_score:
            saved = matchup   
             
    j = PrettyTable()
    j.title = "Worst Loss"
    j.field_names=[saved.home_team.team_name,saved.away_team.team_name]
    j.add_row([saved.home_score, saved.away_score])
    j.align="c"
    print(j)
            

    
worstLoss(league, week)

+----------------------------------+
|            Worst Loss            |
+-----------------------+----------+
| Kupp My  Bells Gurley | El Chapo |
+-----------------------+----------+
|         119.2         |  161.9   |
+-----------------------+----------+


In [276]:
def startingLineupAverage(lineup):
    starterOutput = [] 
    for player in lineup:
        if player.slot_position != "BE" and player.slot_position != "IR":
            starterOutput.append(player.points)         
    return sum(starterOutput)/len(starterOutput)


def highestScoringStarter(lineup):
    highestScore = 0
    bestPlayer = None
    for player in lineup:
        if player.slot_position != "BE" and player.slot_position != "IR":
            if player.points > highestScore:
                highestScore = player.points
                bestPlayer = player
    return bestPlayer
    
    
## Team with greatest differences between max player and team average
def topHeavyTeams(league, week):
    allTeams = []
    box_scores = league.box_scores(week)
    for matchup in box_scores:
        
        ## HOME TEAM
        avg = startingLineupAverage(matchup.home_lineup)
        maxPlayer = highestScoringStarter(matchup.home_lineup)
        allTeams.append((matchup.home_team.team_name, avg, maxPlayer, maxPlayer.points - avg))
        
        ## AWAY TEAM
        avg = startingLineupAverage(matchup.away_lineup)
        maxPlayer = highestScoringStarter(matchup.away_lineup)
        allTeams.append((matchup.away_team.team_name, avg, maxPlayer, maxPlayer.points - avg))
        
    allTeamSorted = sorted(allTeams, key=lambda tup: tup[3], reverse=True)
    return allTeamSorted
                                         
def prettyPrintTopHeavy(topHeavy):
    
    j = PrettyTable()
    j.title = "Top Heavy Teams"
    j.field_names=["Team", "Top Player Pts", "Team Avg", "Diff"]
    for index, team in enumerate(topHeavyList[:3]):
        x="{0}: {1}".format(team[2].name,team[2].points)
        j.add_row([team[0], x, "{:.2f}".format(team[1]), "{:.2f}".format((team[2].points - team[1]))])
    print(j)
    
topHeavyList = topHeavyTeams(league, week)
prettyPrintTopHeavy(topHeavyList)




+---------------------------------------------------------+
|                     Top Heavy Teams                     |
+---------------+----------------------+----------+-------+
|      Team     |    Top Player Pts    | Team Avg |  Diff |
+---------------+----------------------+----------+-------+
| Rosen Sandler | Tua Tagovailoa: 38.7 |  12.12   | 26.58 |
|     Top G     |  Stefon Diggs: 38.8  |  13.07   | 25.73 |
|    El Chapo   | Lamar Jackson: 42.5  |  20.24   | 22.26 |
+---------------+----------------------+----------+-------+


In [269]:
## Team with greatest differences between max player and team average
def highestTeamAverageForStarters(league, week):
    allTeams = []
    box_scores = league.box_scores(week)
    for matchup in box_scores:
        
        ## HOME TEAM
        avg = startingLineupAverage(matchup.home_lineup)
        allTeams.append((matchup.home_team.team_name, avg))
        
        ## AWAY TEAM
        avg = startingLineupAverage(matchup.away_lineup)
        maxPlayer = highestScoringStarter(matchup.away_lineup)
        allTeams.append((matchup.away_team.team_name, avg))
        
    allTeamSorted = sorted(allTeams, key=lambda tup: tup[1], reverse=True)
    return allTeamSorted

def prettyPrintHitters(hitters):
    
    j = PrettyTable()
    j.title = "Best All Around Team"
    j.field_names=["Team", "Avg Points Per Player"]
    
    for team in hitters[:3]:
        j.add_row([team[0], "{:.2f}".format(team[1])])
        
    print(j)
    

hitters = highestTeamAverageForStarters(league,week)
prettyPrintHitters(hitters)                                  

+-----------------------------------------------+
|              Best All Around Team             |
+-----------------------+-----------------------+
|          Team         | Avg Points Per Player |
+-----------------------+-----------------------+
|        El Chapo       |         20.24         |
| Gisele'd in my  Wentz |         16.11         |
|     Grayson BeatYu    |         15.38         |
+-----------------------+-----------------------+


In [296]:
def standings(league, week):
        
    j = PrettyTable()
    x = "JCPY FFL Week {0} Power Rankings".format(league.current_week)
    j.title = x
    
    ### (team_name, total_output, optimal_output, accuracy %)
    total_season_eff = seasonEffiency(league, league.current_week)
    
    
    j.field_names=["Ranking", "Team", "Record", " Yoff Percentage", " Lineup IQ"]
    
    for index, tup in enumerate(league.power_rankings()):
        team = tup[1]
        record = "{0}-{1}-{2}".format(team.wins, team.losses, team.ties)
        
        ## Find Team IQ
        for i in total_season_eff:
            if i[0] == team.team_name:
                
                yoff_pct = "{0:.0f}%".format(team.playoff_pct)
                manager_iq = "{0:.0f}/{1:.0f}: {2:.0f}%".format(i[1],i[2],i[3]*100)
                j.add_row([index+1, team.team_name,record,yoff_pct,manager_iq ])
        
    print(j)
    
standings(league, week)


+---------------------------------------------------------------------------+
|                       JCPY FFL Week 3 Power Rankings                      |
+---------+-----------------------+--------+------------------+-------------+
| Ranking |          Team         | Record |  Yoff Percentage |   Lineup IQ |
+---------+-----------------------+--------+------------------+-------------+
|    1    |     Grayson BeatYu    | 2-0-0  |       86%        | 357/404: 89 |
|    2    |        El Chapo       | 1-1-0  |       67%        | 363/406: 89 |
|    3    |     J.G.'s  ZZ's      | 2-0-0  |       83%        | 299/382: 79 |
|    4    |    Begelbaum Donuts   | 2-0-0  |       73%        | 273/342: 79 |
|    5    | Kupp My  Bells Gurley | 1-1-0  |       77%        | 313/353: 87 |
|    6    |     The  Red Army     | 1-1-0  |       41%        | 262/289: 91 |
|    7    | Gisele'd in my  Wentz | 1-1-0  |       66%        | 269/371: 71 |
|    8    |     Rosen Sandler     | 0-2-0  |       40%        | 

In [264]:
def divison_strength(league, week):
        
    j = PrettyTable()
#     x = "JCPY FFL Week {0} Power Rankings".format(week)
#     j.title = x
#     j.field_names=["Ranking", "Team", "Record", " Yoff Percentage"]
    rankings = []
    east_rankings = []
    west_rankings = []
    for index, tup in enumerate(league.power_rankings()):
        
        team = tup[1]
        ranking = float(tup[0])
        rankings.append(ranking)
        
        if team.division_name == "East":
            east_rankings.append(ranking)
        else:
            west_rankings.append(ranking)
            
    east_strength = sum(east_rankings)
    west_strength =sum(west_rankings)
    
    strong_division = ""
    if east_strength > west_strength:
        strong_division = "East"
        pct_stronger = (1 - (west_strength / east_strength)) * 100
        x = "East Division is {0:.2f}% stronger than West".format(pct_stronger)
    else:
        strong_division = "West"
        pct_stronger = 1 - (east_strength / west_strength).format(pct_stronger) * 100
        x = "West Division is {0:.2f}% stronger than East"

    print(x)
    return x

divison_strength(league, week)


East Division is 14.04% stronger than West


'East Division is 14.04% stronger than West'

In [288]:
############# Final Output #################################


week = 1
## Worst Win
worstWin(league, week)
## Worst Loss
worstLoss(league, week)

## Biggest Blowout
biggestBlowOut(league, week)

## Best and Worst Manager
manager_eff = manager_effiency(league, week)
prettyPrintManagerEff(manager_eff)

## Top Heavy
topHeavyList = topHeavyTeams(league, week)
prettyPrintTopHeavy(topHeavyList)

## Everyone was hitting
hitters = highestTeamAverageForStarters(league,week)
prettyPrintHitters(hitters)                                  

### QBs
list_len = 2
qbWarmers = biggestBenchWarmer(league, week, "QB")
prettyPrintBenchWarmers(qbWarmers[:list_len], "QB")

### RBs
rbWarmers = biggestBenchWarmer(league, week, "RB")
prettyPrintBenchWarmers(rbWarmers[:list_len], "RB")

### TEs
teWarmers = biggestBenchWarmer(league, week, "TE")
prettyPrintBenchWarmers(teWarmers[:list_len], "TE")


#### WRs
teWarmers = biggestBenchWarmer(league, week, "WR")
prettyPrintBenchWarmers(teWarmers[:list_len], "WR")



+--------------------------------------+
|              Worst Win               |
+------------------+-------------------+
| Begelbaum Donuts | Chiraqui Soldiers |
+------------------+-------------------+
|      105.9       |        90.5       |
+------------------+-------------------+
+---------------------------------------+
|               Worst Loss              |
+---------------+-----------------------+
| Rosen Sandler | Kupp My  Bells Gurley |
+---------------+-----------------------+
|     119.3     |         129.0         |
+---------------+-----------------------+
+----------------------------------------+
|            Biggest Blowout             |
+----------------+-----------------------+
| Grayson BeatYu | Gisele'd in my  Wentz |
+----------------+-----------------------+
|     130.3      |          76.4         |
+----------------+-----------------------+
+-------------------------------------+
|     Smartest Managers of the Week   |
+---------------+----------------+----

In [291]:
## Divison Standing
standings(league, week)
divison_strength(league, week)


# prettyPrintSeasonEff(total_season_eff)

+-------------------------------------------------------------+
|                JCPY FFL Week 1 Power Rankings               |
+---------+-----------------------+--------+------------------+
| Ranking |          Team         | Record |  Yoff Percentage |
+---------+-----------------------+--------+------------------+
|    1    |     Grayson BeatYu    | 2-0-0  |       86%        |
|    2    |        El Chapo       | 1-1-0  |       67%        |
|    3    |     J.G.'s  ZZ's      | 2-0-0  |       83%        |
|    4    |    Begelbaum Donuts   | 2-0-0  |       73%        |
|    5    | Kupp My  Bells Gurley | 1-1-0  |       77%        |
|    6    |     The  Red Army     | 1-1-0  |       41%        |
|    7    | Gisele'd in my  Wentz | 1-1-0  |       66%        |
|    8    |     Rosen Sandler     | 0-2-0  |       40%        |
|    9    |         Top G         | 0-2-0  |       34%        |
|    10   |   Chiraqui Soldiers   | 0-2-0  |       33%        |
+---------+-----------------------+-----